In [1]:
import torch
import numpy as np
import pandas as pd
from preprocess import TripPreProcess, ASAPPreProcess
from trainer import RP_trainer
from models import BERT
from utils import set_seed
from transformers import BertModel, BertTokenizer

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_name = 'Trip' # ['ASAP', 'Trip', 'rest_14', 'rest_15', 'rest_16', 'mams']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 1
set_seed(seed)
n_epochs = 10
batch_size = 32

In [3]:
if data_name == 'Trip':
    trip = TripPreProcess()
    T, train_set, dev_set, test_set = trip.get_dataset()
    bert_model = BertModel.from_pretrained("./model_params/bert-base-uncased")
elif data_name == 'ASAP':
    asap = ASAPPreProcess()
    T, train_set, dev_set, test_set = asap.get_dataset()
    bert_model = BertModel.from_pretrained("./model_params/bert-base-chinese")

Some weights of the model checkpoint at ./model_params/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./model_params/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias',

In [4]:
model = BERT(bert_model).to(device)
trainer = RP_trainer()

### Train

In [5]:
trainer.train(model=model, train_set=train_set, dev_set=dev_set, device=device, n_epochs=n_epochs, batch_size=batch_size, data_name=data_name, model_name='BERT_' + str(seed))

  0%|          | 0/734 [00:00<?, ?it/s]/root/miniconda3/envs/myconda/lib/python3.8/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
100%|██████████| 734/734 [02:07<00:00,  5.78it/s]


EPOCH: 1 TRAIN-F1: 0.39927499521333926 TRAIN-LOSS 0.9655647069946621 DEV-F1 0.4924243696046138


100%|██████████| 734/734 [02:03<00:00,  5.92it/s]


EPOCH: 2 TRAIN-F1: 0.46821657716370046 TRAIN-LOSS 0.8644337038172039 DEV-F1 0.6037045406830427


100%|██████████| 734/734 [02:05<00:00,  5.86it/s]


...

EPOCH: 9 TRAIN-F1: 0.6693583007275599 TRAIN-LOSS 0.5899983353015061 DEV-F1 0.6474845607541414


100%|██████████| 734/734 [02:06<00:00,  5.82it/s]


EPOCH: 10 TRAIN-F1: 0.6878992355015671 TRAIN-LOSS 0.5611291193786491 DEV-F1 0.6497498661673011


100%|██████████| 734/734 [02:07<00:00,  5.74it/s]


EPOCH: 11 TRAIN-F1: 0.7046178957709934 TRAIN-LOSS 0.5342976972209288 DEV-F1 0.6550099446981986
Early stopping at epoch 11


### Test

In [6]:
model.load_state_dict(torch.load("./model_params/" + data_name + "_BERT_" + str(seed) + ".model", map_location=device))

<All keys matched successfully>

In [7]:
trainer.test_rp(model, test_set, batch_size=batch_size, device=device)

Precision: 0.6681974972197118
Recall: 0.6685988543568341
F1-score: 0.6669256792440642
Accuracy: 0.7142857142857143
